El objetivo de esta vercion en reducir los outlayers, subir el score y bajar el error

In [114]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import make_pipeline, Pipeline
import xgboost as xgb
import numpy as np

In [90]:
df = pd.read_csv('/Users/klevizane/Documents/CienciaDatos/03_proyectos/02_car_predict price/data/df_with_outlayers.csv')

In [91]:
df

,Make,MSRP,Body Size,Body Style,Engine Aspiration,Drivetrain,Transmission,Horsepower_no,Torque_no
0,Aston Martin,242000.0,Large,SUV,Twin-Turbo,AWD,automatic,697.0,663.0
1,Audi,35800.0,Compact,Sedan,Turbocharged,FWD,automatic,201.0,221.0
2,Audi,37800.0,Compact,Sedan,Turbocharged,AWD,automatic,201.0,221.0
3,Audi,41400.0,Compact,Sedan,Turbocharged,AWD,automatic,201.0,221.0
4,Audi,39400.0,Compact,Sedan,Turbocharged,FWD,automatic,201.0,221.0
...,...,...,...,...,...,...,...,...,...
1605,Nissan,50990.0,Compact,Coupe,Twin-Turbo,RWD,automatic,400.0,350.0
1606,Nissan,53990.0,Compact,Coupe,Twin-Turbo,RWD,automatic,400.0,350.0
1607,Nissan,53990.0,Compact,Coupe,Twin-Turbo,RWD,manual,400.0,350.0
1608,Nissan,40990.0,Compact,Coupe,Twin-Turbo,RWD,manual,400.0,350.0


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1610 entries, 0 to 1609
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Make               1610 non-null   object 
 1   MSRP               1610 non-null   float64
 2   Body Size          1610 non-null   object 
 3   Body Style         1610 non-null   object 
 4   Engine Aspiration  1610 non-null   object 
 5   Drivetrain         1610 non-null   object 
 6   Transmission       1610 non-null   object 
 7   Horsepower_no      1610 non-null   float64
 8   Torque_no          1610 non-null   float64
dtypes: float64(3), object(6)
memory usage: 113.3+ KB


## Separacion de datos

In [93]:
#X = df[['Make', 'Body Size', 'Body Style', 'Engine Aspiration', 'Drivetrain', 'Transmission', 'Horsepower_no', 'Torque_no']] 
X = df.drop('MSRP', axis=1)
y = df['MSRP'].astype(int)


In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size = 0.2, random_state=15)


## Pipe encoding + estandarizado + modelo

Queremos poner en el pipe
- on-hot-enocder = make, body stile, engine aspiration, derivetrain, transmition
- Ordinal encoder = body size
- XGBoost
- escalar los datos

In [95]:
df

,Make,MSRP,Body Size,Body Style,Engine Aspiration,Drivetrain,Transmission,Horsepower_no,Torque_no
0,Aston Martin,242000.0,Large,SUV,Twin-Turbo,AWD,automatic,697.0,663.0
1,Audi,35800.0,Compact,Sedan,Turbocharged,FWD,automatic,201.0,221.0
2,Audi,37800.0,Compact,Sedan,Turbocharged,AWD,automatic,201.0,221.0
3,Audi,41400.0,Compact,Sedan,Turbocharged,AWD,automatic,201.0,221.0
4,Audi,39400.0,Compact,Sedan,Turbocharged,FWD,automatic,201.0,221.0
...,...,...,...,...,...,...,...,...,...
1605,Nissan,50990.0,Compact,Coupe,Twin-Turbo,RWD,automatic,400.0,350.0
1606,Nissan,53990.0,Compact,Coupe,Twin-Turbo,RWD,automatic,400.0,350.0
1607,Nissan,53990.0,Compact,Coupe,Twin-Turbo,RWD,manual,400.0,350.0
1608,Nissan,40990.0,Compact,Coupe,Twin-Turbo,RWD,manual,400.0,350.0


In [96]:
# 1. Listas de Columnas
onh_cols = ['Make', 'Body Style', 'Engine Aspiration', 'Drivetrain', 'Transmission']
ordinal_cols = ['Body Size']
numeric_cols = ["Horsepower_no", "Torque_no"]

# 2. Definir las categorías ordenadas (¡es crucial para OrdinalEncoder!)
body_size = ['Compact', 'Midsize', 'Large']



In [97]:
nominal_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ordinal_transformer = OrdinalEncoder(categories=[body_size])

In [98]:
sclaer = StandardScaler()

In [99]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', sclaer, numeric_cols),
        ('ohe', nominal_transformer, onh_cols),
        ('ord', ordinal_transformer, ordinal_cols)
    ],
    # 'remainder' te permite especificar qué hacer con las columnas no especificadas.
    # 'passthrough' las mantiene sin modificar, 'drop' las elimina.
    remainder='passthrough'
)

In [100]:
# Definir el modelo (p. ej., Regresión Logística)
modelo = xgb.XGBRegressor(random_state=42)

In [137]:
# Crear el pipeline completo: Preprocesamiento + Modelo
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', modelo)])

## Entrenar

In [129]:
pipe.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('ohe', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [130]:
pipe.score(X_test, y_test)
# tenemos un 92% de accuracy super bien vamos a ver los metricas

0.9491128325462341

In [131]:
y_pred = pipe.predict(X_test)


In [ ]:

# 1. R2 (Coeficiente de Determinación)
r2 = r2_score(y_test, y_pred)

# 2. MSE (Error Cuadrático Medio)
mse = mean_squared_error(y_test, y_pred)

# 3. RMSE (Raíz del Error Cuadrático Medio)
# Opción 1: Usando la función de MSE y sqrt
rmse_op1 = sqrt(mse)

# 4. MAE (Error Absoluto Medio)
mae = mean_absolute_error(y_test, y_pred)

# 5. MAPE (Error Porcentual Absoluto Medio)
mape = mean_absolute_percentage_error(y_test, y_pred)

In [112]:
print("--- Métricas de Evaluación de Regresión ---")
print(f"1. R² (Varianza Explicada): {r2:.4f} (Cuanto más cerca de 1.0, mejor)")
print(f"2. RMSE (Error Típico):    ${rmse_op1:,.2f} (Error promedio en unidades de MSRP)")
print(f"3. MAE (Error Absoluto):   ${mae:,.2f} (Error promedio menos sensible a outliers)")
print(f"4. MAPE (Error Porcentual): {mape*100:.2f}% (Error promedio en porcentaje)")

--- Métricas de Evaluación de Regresión ---
1. R² (Varianza Explicada): 0.9491 (Cuanto más cerca de 1.0, mejor)
2. RMSE (Error Típico):    $13,175.89 (Error promedio en unidades de MSRP)
3. MAE (Error Absoluto):   $6,174.78 (Error promedio menos sensible a outliers)
4. MAPE (Error Porcentual): 7.61% (Error promedio en porcentaje)


### Cross val modelo 1

In [133]:
scores_r2 = cross_val_score(
    estimator=pipe,  # Tu pipeline completo
    X=X,              # El DataFrame de características (X) completo
    y=y,              # El vector objetivo (y) completo
    scoring='r2',     # La métrica a usar (R²)
    cv=20              # El número de divisiones (folds)
)

print("--- Validación Cruzada R² ---")
print(f"Scores individuales por fold (R²): {scores_r2}")
print(f"R² promedio de CV: {scores_r2.mean():.4f} (+/- {scores_r2.std() * 2:.4f} desv. est.)")

--- Validación Cruzada R² ---
Scores individuales por fold (R²): [ 0.8923834   0.61308914  0.9502852   0.97674203  0.4321937   0.77885246
  0.82509452  0.933851    0.96688342  0.66856694  0.9610005   0.96673226
  0.97159076 -2.36412978  0.7878412   0.53068233  0.67718107  0.89231831
  0.88453567  0.95046639]
R² promedio de CV: 0.6648 (+/- 1.4249 desv. est.)


In [134]:
# Para MAE, usamos la métrica negativa (porque sklearn maximiza)
scores_mae_neg = cross_val_score(
    estimator=pipe,
    X=X,
    y=y,
    scoring='neg_mean_absolute_error', # Devuelve un valor NEGATIVO
    cv=20
)

# Convertir de vuelta a MAE positivo para interpretación
scores_mae = -scores_mae_neg

print("\n--- Validación Cruzada MAE ---")
print(f"Scores MAE por fold: ${scores_mae.mean():,.2f} (+/- ${scores_mae.std() * 2:,.2f} desv. est.)")


--- Validación Cruzada MAE ---
Scores MAE por fold: $7,223.59 (+/- $5,715.44 desv. est.)


## Modelo 2 prubea

In [141]:
scorer_a_minimizar = 'neg_mean_absolute_error'
param_grid = {
    # La clave ahora es 'regressor__', coincidiendo con el nombre del paso.
    
    # Parámetros de Boosting
    'regressor__n_estimators': [100, 300], 
    'regressor__learning_rate': [0.05, 0.1],

    # Parámetros de la Estructura del Árbol
    'regressor__max_depth': [3, 5],
    'regressor__min_child_weight': [1, 3],
    
    # Parámetros de Submuestreo
    'regressor__colsample_bytree': [0.7, 1.0], 
}

In [142]:
grid_search = GridSearchCV(
    estimator=pipe,           # El pipeline que contiene el preprocesador y el modelo
    param_grid=param_grid,     # El diccionario con los hiperparámetros
    scoring=scorer_a_minimizar, # La métrica de evaluación (neg_MAE)
    cv=3,                      # Número de folds para la validación cruzada (usar 3 para rapidez)
    verbose=1,                 # Muestra el progreso
    n_jobs=-1                  # Usa todos los núcleos de tu CPU
)

# 2. Ejecutar la búsqueda
# ¡Aquí usas X_train y y_train, no el conjunto completo!
# El CV interno se encarga de la validación.
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


,estimator,"Pipeline(step...=None, ...))])"
,param_grid,"{'regressor__colsample_bytree': [0.7, 1.0], 'regressor__learning_rate': [0.05, 0.1], 'regressor__max_depth': [3, 5], 'regressor__min_child_weight': [1, 3], ...}"
,scoring,'neg_mean_absolute_error'
,n_jobs,-1
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('ohe', ...), ...]"


In [143]:
# 1. Obtener los mejores parámetros
print("\n--- Resultados Óptimos ---")
print("Mejores Hiperparámetros (para 'regressor'):")
print(grid_search.best_params_)

# 2. Obtener y revertir el mejor score (MAE negativo)
mejor_mae_cv = -grid_search.best_score_
print(f"\nMejor MAE Promedio (Validación Cruzada): ${mejor_mae_cv:,.2f}")

# 3. Evaluar el mejor modelo en el conjunto de prueba (Test set)
mejor_pipeline = grid_search.best_estimator_ # Este es el mejor pipeline re-ajustado

# Hacer predicciones en el conjunto de prueba
y_pred_final = mejor_pipeline.predict(X_test)
mae_final_test = mean_absolute_error(y_test, y_pred_final)

print(f"\nMAE Final en el Conjunto de Prueba (X_test): ${mae_final_test:,.2f}")


--- Resultados Óptimos ---
Mejores Hiperparámetros (para 'regressor'):
{'regressor__colsample_bytree': 0.7, 'regressor__learning_rate': 0.1, 'regressor__max_depth': 5, 'regressor__min_child_weight': 1, 'regressor__n_estimators': 300}

Mejor MAE Promedio (Validación Cruzada): $7,039.65

MAE Final en el Conjunto de Prueba (X_test): $6,039.13


## Modelo final

In [144]:
mejores_params = {'regressor__colsample_bytree': 0.7, 
                  'regressor__learning_rate': 0.1, 
                  'regressor__max_depth': 5, 
                  'regressor__min_child_weight': 1, 
                  'regressor__n_estimators': 300}

params_limpios = {k.replace('regressor__', ''): v for k, v in mejores_params.items()}

In [145]:
modelo_xgb_optimo = xgb.XGBRegressor(
    random_state=42, 
    **params_limpios # Desempaqueta el diccionario de parámetros limpios
)

# 2. Crear el pipeline final (usando el preprocessor original)
pipe_final = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', modelo_xgb_optimo)
])


In [146]:
pipe_final.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('ohe', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [152]:
import joblib
joblib.dump(pipe_final, "02_modelxgb.pkl")

['02_modelxgb.pkl']